## My first playground (Copy this and do not work on a teammate's notebook)

Always make sure that you copy and name the **notebook prefixed with your GitHub username**, eg. g0zzy_data_viz.ipynb, g0zzy_feature_eng.ipynb, etc.

Create as many as needed per task, trying to keep the notebook coherent: Do not chuck all the tasks into one where you have the data cleaning, model, fastapi, frontend in one :D 

Always remove unnecessary things from your notebook so that you do not confuse teammates ;) 

In [6]:
import pandas as pd

# Loading the dataset
path = "/Users/gozde/code/g0zzy/stress_sense/raw_data/Data.csv"
data = pd.read_csv(path)

data.head()

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety


In [15]:
data.shape

(53043, 2)

In [10]:
data.statement.isnull().value_counts()

statement
False    52681
True       362
Name: count, dtype: int64

In [12]:
data[data.statement.isnull()].head(20)

,Unnamed: 0,statement,status
293,293,NaN,Anxiety
572,572,NaN,Anxiety
595,595,NaN,Anxiety
1539,1539,NaN,Normal
2448,2448,NaN,Normal
2470,2470,NaN,Normal
2753,2753,NaN,Normal
4331,4331,NaN,Normal
5878,5878,NaN,Normal
6513,6513,NaN,Normal


In [13]:
data.drop(columns=["Unnamed: 0"], inplace=True)

In [14]:
data.head()

,statement,status
0,oh my gosh,Anxiety
1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,I've shifted my focus to something else but I'...,Anxiety
4,"I'm restless and restless, it's been a month n...",Anxiety


In [16]:
data.dropna(inplace=True)

In [17]:
data.shape

(52681, 2)

In [18]:
data.drop_duplicates(inplace=True)

In [19]:
data.shape

(51093, 2)

In [21]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [22]:
import string
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

def preprocessing(sentence):
    # Removing whitespaces
    sentence = sentence.strip()

    # Lowercasing
    sentence = sentence.lower()

    # Removing numbers
    sentence = ''.join(char for char in sentence if not char.isdigit())

    # Removing punctuation
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '')

    # Tokenizing
    tokenized = word_tokenize(sentence)

    # Lemmatizing
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in tokenized]
    cleaned_sentence = " ".join(lemmatized)
    return cleaned_sentence

In [24]:
data['clean_statement'] = data.statement.apply(preprocessing)

In [25]:
data.head(20)

,statement,status,clean_statement
0,oh my gosh,Anxiety,oh my gosh
1,"trouble sleeping, confused mind, restless hear...",Anxiety,trouble sleeping confused mind restless heart ...
2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety,all wrong back off dear forward doubt stay in ...
3,I've shifted my focus to something else but I'...,Anxiety,ive shifted my focus to something else but im ...
4,"I'm restless and restless, it's been a month n...",Anxiety,im restless and restless it been a month now b...
5,"every break, you must be nervous, like somethi...",Anxiety,every break you must be nervous like something...
6,"I feel scared, anxious, what can I do? And may...",Anxiety,i feel scared anxious what can i do and may my...
7,Have you ever felt nervous but didn't know why?,Anxiety,have you ever felt nervous but didnt know why
8,"I haven't slept well for 2 days, it's like I'm...",Anxiety,i havent slept well for day it like im restles...
9,"I'm really worried, I want to cry.",Anxiety,im really worried i want to cry


In [26]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Split
X_train, X_test, y_train, y_test = train_test_split(
    data["clean_statement"], data["status"], test_size=0.2, stratify=data["status"], random_state=42
)

# TF-IDF features
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Classifier
clf = LogisticRegression(max_iter=500, class_weight="balanced")
clf.fit(X_train_tfidf, y_train)

# Evaluate
y_pred = clf.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

                      precision    recall  f1-score   support

             Anxiety       0.73      0.79      0.76       725
             Bipolar       0.69      0.75      0.72       500
          Depression       0.77      0.56      0.65      3019
              Normal       0.90      0.92      0.91      3208
Personality disorder       0.35      0.59      0.44       179
              Stress       0.43      0.67      0.52       459
            Suicidal       0.66      0.74      0.70      2129

            accuracy                           0.74     10219
           macro avg       0.65      0.72      0.67     10219
        weighted avg       0.76      0.74      0.74     10219



Precision = out of predicted positives, how many were correct?

Recall = out of actual positives, how many were caught?

F1 = harmonic mean of precision & recall → balances the two.

Stress posts → we care about recall (don’t miss them).